In [1]:
import pandas as pd

from GoogleNews import GoogleNews
import tweepy
import re
#import required libraries
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize

In [2]:
keywordsList = ['e-commerce','ecommerce','Ecommerce','eCommerce','retail']

searchTextList = []
sourceList = []
titleList = []
mediaList = []
dateList = []
descList = []
linkList = []
detailedDescList = []

In [3]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
googlenews = GoogleNews()
googlenews.setlang('en')
googlenews.setperiod('d')
googlenews.setTimeRange('06/01/2020','09/26/2020')
googlenews.setencode('utf-8')

In [5]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [6]:
for searchText in keywordsList:
    print(searchText)
    googlenews.search(searchText)
    newsResults = googlenews.result()
    df = pd.DataFrame(newsResults)
    for index, row in df.iterrows(): 
        #searchTextList.append(searchText)
        sourceList.append("GoogleNews")
        titleList.append(row["title"])
        mediaList.append(row["media"])
        dateList.append(row["date"])
        descList.append(row["desc"])
        linkList.append(row["link"])
        detailedDescList.append(url_to_string(str(row["link"])))
    

e-commerce
ecommerce
Ecommerce
eCommerce
retail


In [7]:
def get_text_for_tweet(id):
    status = api.get_status(id)
    return status.text.encode("utf-8")

In [8]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [10]:
date_since = "2020-06-01"
date_until = "2020-09-26"
for search_term in keywordsList:
    # Creation of query method using parameters
    for tweet in tweepy.Cursor(api.search, 
            q='\"{}\" -filter:retweets'.format(search_term), 
            count=100, lang='en', tweet_mode='extended', since=date_since).items(100):
        #searchTextList.append(searchText)
        sourceList.append("Twitter")
        titleList.append(get_text_for_tweet(tweet.id))
        mediaList.append(tweet.source)
        dateList.append(tweet.created_at)
        descList.append(clean_tweet(tweet.full_text))
        linkList.append(tweet.source_url)
        detailedDescList.append(clean_tweet(tweet.full_text))

In [11]:
main_df = pd.DataFrame(list(zip(sourceList, titleList, mediaList, dateList, descList, linkList, detailedDescList)), 
               columns =['Source', 'Title', 'Media', 'Date', 'Description', 'Link', 'Detailed Description']) 
main_df.to_excel("NLP_Assignment_PostLockdown.xlsx")  

In [ ]:
main_df.head()